<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_AGENT_0DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://mistral.ai/news/agents-api

https://console.mistral.ai/usage

In [ ]:
!pip install colab-env --quiet
!pip install mistralai --quiet

In [ ]:
from IPython import get_ipython
from IPython.display import display
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env

In [6]:
from mistralai import Mistral
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)


client = Mistral(api_key=api_key)  # Use MistralClient instead of Mistral
model_list = client.models.list()

for model in model_list.data:
    print(model.id)
    #print(model.created)
    #print(model.owned_by)

ministral-3b-2410
ministral-3b-latest
ministral-8b-2410
ministral-8b-latest
open-mistral-7b
mistral-tiny
mistral-tiny-2312
open-mistral-nemo
open-mistral-nemo-2407
mistral-tiny-2407
mistral-tiny-latest
open-mixtral-8x7b
mistral-small
mistral-small-2312
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2409
mistral-large-2407
mistral-large-2402
mistral-large-2411
mistral-large-latest
pixtral-large-2411
pixtral-large-latest
mistral-large-pixtral-2411
codestral-2501
codestral-latest
codestral-2412
codestral-2411-rc5
codestral-2405
devstral-small-2505
devstral-small-latest
pixtral-12b-2409
pixtral-12b
pixtral-12b-latest
mistral-small-2501
mistral-small-2503
mistral-small-latest
mistral-small-2402
mistral-saba-2502
mistral-saba-latest
mistral-medium-2505
mistral-medium-latest
mistral-medium
mistral-medium-2312
magistral-medium-2506
magistral-medium-latest
magistral-small-2506
magistral-small-latest
mistral-embed
codestral-embed
codestral-embed-2505
mistral-moderation-2411
mistral-mod

In [7]:
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env --quiet
    import colab_env

try:
    import mistralai
    # Re-import after upgrade to ensure new version is loaded in current session if needed
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai --upgrade'.")
    exit()

# %%
# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# --- Agent Definitions ---
# Pydantic model for Calculator Agent's response format (still relevant if that agent is used)
class CalcResult(BaseModel):
    reasoning: str
    result: str

print("Creating AI agents...")

finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent used to answer financial related requests",
    name="finance-agent",
)
web_search_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent that can search online for any information if needed",
    name="websearch-agent",
    tools=[{"type": "web_search"}],
)
ecb_interest_rate_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Can find the current interest rate of the European central bank",
    name="ecb-interest-rate-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_european_central_bank_interest_rate",
                "description": "Retrieve the real interest rate of European central bank.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                        },
                    },
                    "required": [
                        "date",
                    ]
                },
            },
        },
    ],
)
graph_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="graph-drawing-agent",
    description="Agent used to create graphs using the code interpreter tool.",
    instructions="Use the code interpreter tool when you have to draw a graph.",
    tools=[{"type": "code_interpreter"}]
)
calculator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
          "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

# --- Custom function for flight search (MOCK) ---
# This is a LOCAL MOCK FUNCTION.
# When the remote Mistral AI model proposes to use this tool,
# this local Python function needs to be executed by your application.
# For a real integration, this functionality would typically be
# an actual web service endpoint accessible by Mistral's platform.
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):
    """
    MOCK: This function simulates searching for available flights.
    It returns dummy data based on specific input parameters.
    """
    print(f"\n[DEBUG] MOCK CALL: search_flights with origin='{origin}', destination='{destination}', departure_date='{departure_date}', return_date='{return_date}'")

    origin_norm = origin.upper() if origin else ""
    destination_norm = destination.upper() if destination else ""

    # Dummy data for demonstration
    if origin_norm == "YUL" and destination_norm == "JFK" and departure_date == "2025-07-01":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"},
                {"flight_number": "DL123", "airline": "Delta Airlines", "departure_time": "11:00 AM", "arrival_time": "12:45 PM", "price": "$280 CAD"}
            ]
        }
    elif origin_norm == "LAX" and destination_norm == "SFO" and departure_date == "2025-08-15":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "UA456", "airline": "United Airlines", "departure_time": "09:00 AM", "arrival_time": "10:15 AM", "price": "$120 USD"},
            ]
        }
    else:
        return {"flights_found": False, "details": "No flights found for the specified criteria."}

# --- Flight Planning Agent Definition ---
# The agent is defined with its capabilities and tools on the Mistral platform.
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with flight planning requests, searching for flights and providing relevant information. Use IATA airport codes for origin and destination if possible. For example, Montreal is YUL and New York is JFK or LGA.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                        "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                        "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        # For agent creation, this simple format for web_search is usually correct.
        {"type": "web_search"}
    ]
)

print(f"\nFlight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")
tool_names_list = []
for tool in flight_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list.append(tool.function.name)
    else:
        tool_names_list.append(tool.type)
print(f"Tools available to flight_planning_agent: {tool_names_list}")


# --- Test Case Execution for Flight Planning Agent ---
print("\n--- Executing Test Case for the Flight Planning Agent (via chat completions) ---")
print("This simulates a multi-turn conversation where your code acts as the tool executor.")

# Initialize conversation history
conversation_history = []
user_query = "Find me a flight from Montreal to New York on July 1st, 2025."
print(f"\nUser: {user_query}")
# Using standard dictionary for message, as per your working reference
conversation_history.append({"role": "user", "content": user_query})

try:
    # Manual construction of tools list for the chat.complete call
    # This bypasses potential serialization issues from agent.tools objects
    # and ensures the exact dictionary format the API expects.
    # CRITICAL FIX: The error "Input should be 'function'" for 'web_search' implies that
    # the 'type' field for web_search MUST also be 'function' in this context.
    api_call_tools_list = [
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                        "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                        "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        # WORKAROUND: Force 'web_search' to have 'type: "function"' and a dummy 'function' definition
        # to satisfy the API validator, even though it's typically just {"type": "web_search"}.
        {
            "type": "function", # <-- Changed from "web_search" to "function"
            "function": {
                "name": "internal_web_search_tool", # A dummy function name
                "description": "Accesses the internet to find information.",
                "parameters": { # Parameters are usually dynamic for web_search, so empty object
                    "type": "object",
                    "properties": {}
                }
            }
        }
    ]

    # First turn: Send user query. The agent (model) will decide if it needs a tool.
    print("[DEBUG] Sending initial user query to the agent...")
    response_turn1 = client.chat.complete( # Using client.chat.complete as requested
        model=flight_planning_agent.model, # Use the model associated with the agent
        messages=conversation_history,
        tools=api_call_tools_list, # Provide the manually formatted tools
        # tool_choice="auto" is the default when tools are provided
    )

    assistant_message_turn1 = response_turn1.choices[0].message
    # Convert SDK Message object to dictionary before appending to history
    conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1.__dict__)

    if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
        print("\nAgent proposed tool calls (Turn 1):")
        for tool_call in assistant_message_turn1.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")

            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args) # Execute local mock function
                    tool_output_content = json.dumps(tool_output) # Convert output to JSON string
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})

            # Handle the web_search tool, now with its dummy 'function' name
            elif tool_call.function.name == "internal_web_search_tool": # Use the dummy name defined in api_call_tools_list
                print(f"  [DEBUG] Web search requested by agent. Providing mock output for local simulation.")
                tool_output_content = json.dumps("Mock web search: Found airport codes (YUL, JFK) and general flight information resources related to flights.")
            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            # Add the tool output message to the conversation history
            # This is crucial for the model to "see" the result of the tool call
            conversation_history.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name, # Use the name of the tool called
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id # Link to the specific tool call request
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

        # Second turn: Send the conversation history (including tool outputs) back to the model
        # The model will then generate a final response based on the tool's output.
        print("\n[DEBUG] Sending conversation history with tool outputs back for final response...")
        final_response = client.chat.complete( # Using client.chat.complete
            model=flight_planning_agent.model,
            messages=conversation_history,
            tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context
        )

        final_assistant_message = final_response.choices[0].message
        print("\nAgent's Final Response:")
        print(final_assistant_message.content)
        # Convert SDK Message object to dictionary before appending to history
        conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message.__dict__)
    else:
        # If no tool calls were proposed in the first turn, print the direct response
        print("\nAgent's initial response (no tool calls proposed):")
        print(assistant_message_turn1.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction: {e}")
    print("Please check your API key, model availability, network connection, or SDK version.")
    print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) after `pip install --upgrade` is highly recommended.")


print("\n--- Test case execution complete. ---")

Creating AI agents...

Flight Planning Agent 'flight-planning-agent' created with ID: ag_06856578387f719f8000ee59af2bd5ca
Tools available to flight_planning_agent: ['search_flights', 'web_search']

--- Executing Test Case for the Flight Planning Agent (via chat completions) ---
This simulates a multi-turn conversation where your code acts as the tool executor.

User: Find me a flight from Montreal to New York on July 1st, 2025.
[DEBUG] Sending initial user query to the agent...

Agent proposed tool calls (Turn 1):
  Tool Name: search_flights
  Tool Arguments (JSON string): {"origin": "YUL", "destination": "JFK", "departure_date": "2025-07-01"}

[DEBUG] MOCK CALL: search_flights with origin='YUL', destination='JFK', departure_date='2025-07-01', return_date='None'
  [DEBUG] Local MOCK search_flights executed. Output: {"flights_found": true, "details": [{"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"}, {"fli